# **VERSION 6**

---



##**A) pre-work segment**

###**1. download and install modules**

In [31]:
!pip install ConfigParser

In [ ]:
#pip install googlemaps

###**2. mount drive**

In [32]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


###**3. import needed classes and methods**

In [33]:
import configparser
from textblob import TextBlob
import tweepy
import matplotlib.pyplot as plt
import regex as re
import spacy
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
#import googlemaps
#import sys
#import os
import pandas as pd

###**4. download from imported modules**

In [34]:
nltk.download("vader_lexicon")

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

###**5. connect & authenticate**

In [35]:
!ls '/content/drive/My Drive/Colab Notebooks/uemprojectgroup covid_1.properties'

'/content/drive/My Drive/Colab Notebooks/uemprojectgroup covid_1.properties'


In [36]:
config=configparser.RawConfigParser()
config.read('/content/drive/My Drive/Colab Notebooks/uemprojectgroup covid_1.properties')
print(config.sections())

['twitter', 'gmaps']


In [37]:
consumerKey=config.get('twitter','consumerKey')
consumerSecret=config.get('twitter','consumerSecret')
accessToken=config.get('twitter','accessToken')
accessTokenSecret=config.get('twitter','accessTokenSecret')

In [38]:
#print(consumerKey,"\n",consumerSecret,"\n",accessToken,"\n",accessTokenSecret)

In [39]:
auth=tweepy.OAuthHandler(consumer_key=consumerKey,consumer_secret=consumerSecret)
auth.set_access_token(accessToken, accessTokenSecret)
api=tweepy.API(auth,wait_on_rate_limit=True,wait_on_rate_limit_notify=True)

In [40]:
#gmaps=googlemaps.Client(key=config.get('gmaps','gApiKey'))

##**B) methods for work segment**

###**1. send tweet**

In [41]:
#while True:
#  api=tweepy.API(auth,wait_on_rate_limit=True,wait_on_rate_limit_notify=True)

In [42]:
#tw=tweepy.Cursor(api.search, q="covid"+"-filter:retweets",lang="English").items(100)
#for i in tw:
#  print([i.text,i.user.screen_name,i.user.location,i.geo])

In [43]:
def sendTweet():
  tweet=input("enter your tweet: ")
  api.update_status(tweet)

###**2. create dataset with specific keyword**

In [44]:
def searchTweetWithKeyword():

  covid_list=[]


  searchTerm,noOfRecords=input("enter keyword and max no. of records needed:").split()
  noOfRecords=int(noOfRecords)

  #a_tweets=tweepy.Cursor(api.search, q=searchTerm+"-filter:retweets",lang="English").items(noOfRecords)
  a_tweets=tweepy.Cursor(api.search, q=searchTerm+"-filter:retweets").items(noOfRecords)

  for i in a_tweets:
    covid_list.append([i.text,i.user.screen_name,i.user.location,i.geo])


  covid_df=pd.DataFrame(covid_list,columns=["tweets","user_name","location","geo"])
  totalNoOfRecords=len(covid_df)

  print("total no. of records: ",totalNoOfRecords)
  print("records: ")
  display(covid_df)

  return covid_df,totalNoOfRecords

###**3. check tweet sentiments using TextBlob**

In [45]:
def percentage(part,whole):
  return part*100/whole


In [46]:
def checkSentimentTextBlob(covid_df,noOfRecords):
  positive=0
  negative=0
  neutral=0
  polarity=0

  for i in range(noOfRecords):
    textblob_text=TextBlob(covid_df.iloc[i,0])
    value=textblob_text.sentiment.polarity
    polarity+=value
    
    if value<0:
        negative+=1
    elif value==0:
        neutral+=1
    elif value>0:
        positive+=1

  print("raw numbers:\npositive=",positive,"\nnegative=",negative,"\nneutral=",neutral,"\npolarity=",polarity)
  
  positive=format(percentage(positive,noOfRecords),'.2f')
  negative=format(percentage(negative,noOfRecords),'.2f')
  neutral=format(percentage(neutral,noOfRecords),'.2f')
  polarity=format(percentage(polarity,noOfRecords),'.2f')

  print("percentage:\npositive=",positive,"\nnegative=",negative,"\nneutral=",neutral,"\npolarity=",polarity)

###**4. count tweet from each location (city)**

In [47]:
def tweetLocations(covid_df,noOfRecords):
  
  display(covid_df['location'].value_counts())

###**5. clean tweets**

In [48]:
def cleanTweets(covid_df,noOfRecords):
  for i in range(noOfRecords):
    text=covid_df.iloc[i,0]
    
    text=re.sub("@[\w]*","",text)
    text=re.sub("RT @[\w]*:","",text)
    text=re.sub("https?://[a-zA-Z0-9./]*","",text)
    text=re.sub("\n","",text)

    covid_df.iloc[i,0]=text
  
  display(covid_df)
  return covid_df,noOfRecords

###**6. convert dataframe to csv**

In [49]:
def toCSV(covid_df):
  covid_df.to_csv('covid_csv.csv')

###**7. show entites in text**

In [50]:
def showEntities(covid_df,noOfRecords):
  nlp=spacy.load('en_core_web_sm')

  covid_df['entities']=covid_df['tweets'].apply(lambda tweet: [(ent.text,ent.label_) if (not ent.text.startswith('#')) else "" for ent in nlp(tweet).ents])
  
  display(covid_df)
  return covid_df,noOfRecords

###**8. check tweet sentiments using Sentiment Intensity Analyzer**

In [51]:
def tweetSentimentIntensityAnalyzer(covid_df,noOfRecords):
  sid=SentimentIntensityAnalyzer()
  covid_df['sentiment']=covid_df['tweets'].apply(lambda tweet: sid.polarity_scores(tweet))
  display(covid_df)
  return covid_df,noOfRecords 

###**9. categorizing by location using googlemaps api**

In [52]:
#not working till we set payment details 

In [53]:
def getCountry(location):
  result=gmaps.geocode(location)
  result=(result[0]['formatted_address'].split(','))[-1].strip()
  return result

In [54]:
def tweetLocationsUsingGmaps(covid_df,noOfRecords):
  covid_df['country']=covid_df['location'].apply(lambda location: "" if(not location.strip() ) else getCountry(location))

  print(covid_df['country'].value_counts())

  return covid_df,noOfRecords


###**10. filter tweets in dataset by keywords**

In [55]:
def search(keyword,tweets):
  temp=[]
  for t in tweets:
    #print(type(t))
    #print(t)
    #print(re.search(keyword,t))
    if re.search(keyword.lower(),t.lower()):
      temp.append(True)
    else:
      temp.append(False)
  return temp

In [56]:
def filterDataSet(covid_df,noOfRecords):
  
  while True:
    searchTerm=input("enter keyword:")
    #covid_df=covid_df[ lambda covid_df.tweets,searchTerm: return re.search(keyword,text) ]
    ncovid_df=covid_df[ search(searchTerm,covid_df.tweets) ]
    display(ncovid_df)
    ch1=input("do you want to keep the changes?\n y or n:")
    if  (ch1=='y' or ch1=='Y'):
      covid_df=ncovid_df
      

    ch2=input("do you want to search for another keyword from within your dataset?\n y or n:")
    if not (ch2=='y' or ch2=='Y'):
      break

  print("FINAL DATASET:")
  totalNoOfRecords=len(covid_df)
  print("no. of records:",totalNoOfRecords)
  print("records:")
  display(covid_df)
  return covid_df,totalNoOfRecords

##**C) main method segment**

In [57]:
def main():
  covid_df=None
  noOfRecords=None

  print(
  "press 0 to display dataframe\n"
  "press 1 to send tweet\n"
  "press 2 to find tweets with keyword\n"
  "press 3 to check sentiments using TextBlob\n"
  "press 4 to categorize and count by location (city)\n"
  "press 5 to clean tweets\n"
  "press 6 to covert dataframe into csv\n"
  "press 7 to show entities in text\n"
  "press 8 to check tweet sentiments using Sentiment Intensity Analyzer\n"
  "press 9 to categorize and count by location (country) using googlemaps api\n"
  "press 10 to filter tweets in dataset by keywords"
  "press 100 to exit\n")

  while True:
    ch=int(input("choose: "))
    if ch==0:
        display(covid_df)
    elif ch==1:
      sendTweet()
    elif ch==2:
      covid_df,noOfRecords=searchTweetWithKeyword()
    elif ch==3:
      checkSentimentTextBlob(covid_df,noOfRecords)
    elif ch==4:
      tweetLocations(covid_df,noOfRecords)
    elif ch==5:
      covid_df,noOfRecords=cleanTweets(covid_df,noOfRecords)
    elif ch==6:
      toCSV(covid_df)
    elif ch==7:
      covid_df,noOfRecords=showEntities(covid_df,noOfRecords)
    elif ch==8:
      covid_df,noOfRecords=tweetSentimentIntensityAnalyzer(covid_df,noOfRecords)
    elif ch==9:
      covid_df,noOfRecords=tweetLocationsUsingGmaps(covid_df,noOfRecords)
    elif ch==10:
      covid_df,noOfRecords=filterDataSet(covid_df,noOfRecords)
    elif ch==100:
      break
    else:
      print("INVALID CHOICE !")

In [58]:
if __name__=="__main__":
 main()

press 0 to display dataframe
press 1 to send tweet
press 2 to find tweets with keyword
press 3 to check sentiments using TextBlob
press 4 to categorize and count by location (city)
press 5 to clean tweets
press 6 to covert dataframe into csv
press 7 to show entities in text
press 8 to check tweet sentiments using Sentiment Intensity Analyzer
press 9 to categorize and count by location (country) using googlemaps api
press 10 to filter tweets in dataset by keywordspress 100 to exit

choose: 1
enter your tweet: testing v6
choose: 2
enter keyword and max no. of records needed:covid 500
total no. of records:  500
records: 


,tweets,user_name,location,geo
0,"Gawat! Kasus Infeksi Covid-19 Tembus 100.000, ...",motorplusweekly,"Jakarta, Indonesia",None
1,@firdzaradiany Emangnya karyawan test covid-19...,TeamYuigahama,Jakarta Indonesia,None
2,My friend lost her father to COVID-19. Please ...,bane_cc,,None
3,"CG PPHT 2020: Exam (Cancelled), Admit Card, Pa...",Eduportplus,,None
4,@rafetavlc @pablocasado_ Pablo Casado pide man...,AndrsGa08263745,,None
...,...,...,...,...
495,Badan Kebijakan Fiskal akan mengadakan Call Fo...,himapajakfiaub,Malang - Indonesia,None
496,"No compartir utensilios ni platos, usar la mas...",Adeces,España,None
497,@repjerrynadler @tedwheeler do they have a per...,JJChronicles,,None
498,The Secretary of State for Health Matt Hancock...,aquarius_sonia,UK,None


choose: 3
raw numbers:
positive= 105 
negative= 82 
neutral= 313 
polarity= 4.646969922438669
percentage:
positive= 21.00 
negative= 16.40 
neutral= 62.60 
polarity= 0.93
choose: 4


                       155
India                    8
London                   4
New York, NY             4
Australia                4
                      ... 
Харків                   1
brighton                 1
West Cornwall, UK        1
Glasgow                  1
Hessen Deutschland       1
Name: location, Length: 291, dtype: int64

choose: 5


...

do you want to keep the changes?
 y or n:y
do you want to search for another keyword from within your dataset?
 y or n:n
FINAL DATASET:
no. of records: 6
records:


,tweets,user_name,location,geo,entities,sentiment
101,India has prepared five sites for the final hu...,NewsBytesApp,"Gurgaon, Haryana",None,"[(India, GPE), (five, CARDINAL), (Oxfor, NORP)]","{'neg': 0.0, 'neu': 0.905, 'pos': 0.095, 'comp..."
150,India’s COVID Mortality rate is the lowest i...,7notesndreams,,None,"[(India, GPE), (COVID Mortality, ORG)]","{'neg': 0.14, 'neu': 0.86, 'pos': 0.0, 'compou..."
347,#HelphenIndia #lockdown #COVID #mumbai #storie...,HelphenIndia,"Tamil Nadu, India",None,"[(HelphenIndia, PERSON)]","{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
379,India in better position in fighting COVID 19 ...,NishaShekh,Earth,None,"[(India, GPE), (COVID, ORG), (Modi😒, PERSON)]","{'neg': 0.15, 'neu': 0.601, 'pos': 0.249, 'com..."
433,"""We have completed 2,000 #VandeBharat evacuati...",MahtabNama,"Dehli, from #Supaul, Bihar",None,"[(2,000 #VandeBharat, MONEY), (6.7, CARDINAL),...","{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
452,"""COVID-19 in India"" #coronavirus",Sainidan1,"Barmer, India",None,"[(India, GPE)]","{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."


choose: 0


,tweets,user_name,location,geo,entities,sentiment
101,India has prepared five sites for the final hu...,NewsBytesApp,"Gurgaon, Haryana",None,"[(India, GPE), (five, CARDINAL), (Oxfor, NORP)]","{'neg': 0.0, 'neu': 0.905, 'pos': 0.095, 'comp..."
150,India’s COVID Mortality rate is the lowest i...,7notesndreams,,None,"[(India, GPE), (COVID Mortality, ORG)]","{'neg': 0.14, 'neu': 0.86, 'pos': 0.0, 'compou..."
347,#HelphenIndia #lockdown #COVID #mumbai #storie...,HelphenIndia,"Tamil Nadu, India",None,"[(HelphenIndia, PERSON)]","{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
379,India in better position in fighting COVID 19 ...,NishaShekh,Earth,None,"[(India, GPE), (COVID, ORG), (Modi😒, PERSON)]","{'neg': 0.15, 'neu': 0.601, 'pos': 0.249, 'com..."
433,"""We have completed 2,000 #VandeBharat evacuati...",MahtabNama,"Dehli, from #Supaul, Bihar",None,"[(2,000 #VandeBharat, MONEY), (6.7, CARDINAL),...","{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
452,"""COVID-19 in India"" #coronavirus",Sainidan1,"Barmer, India",None,"[(India, GPE)]","{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."


choose: 100




---


# **END OF CODE, ROUGH WORK BEGINS:**


---



In [59]:
print(__name__)

__main__


In [60]:
def create_df():
  dfr=pd.DataFrame([['qwerty','be','mumbai, india','d','e'],['uiop','be','canada',None,'e'],['wasd','bi','dallas tx','d','e'],['fghjklzxc','b','european union','d','e']],columns=['A','B','C','D','E'])
  display("records:",dfr)
  return dfr

In [61]:
dfr=create_df()

'records:'

,A,B,C,D,E
0,qwerty,be,"mumbai, india",d,e
1,uiop,be,canada,None,e
2,wasd,bi,dallas tx,d,e
3,fghjklzxc,b,european union,d,e


In [62]:
dfr1=pd.DataFrame([['zxcasdedc','bo','kolkata, india','de','ei'],['uiop','be','orissa','dio','eu']],columns=['A','B','C','D','E'])
dfr=dfr.append(dfr1,ignore_index=True)
dfr.drop_duplicates(inplace=True)
dfr

#df.iloc[:,[1,3]]

,A,B,C,D,E
0,qwerty,be,"mumbai, india",d,e
1,uiop,be,canada,None,e
2,wasd,bi,dallas tx,d,e
3,fghjklzxc,b,european union,d,e
4,zxcasdedc,bo,"kolkata, india",de,ei
5,uiop,be,orissa,dio,eu


In [63]:
dfr.iloc[3,2]

'european union'

In [64]:
dfr['A']

0       qwerty
1         uiop
2         wasd
3    fghjklzxc
4    zxcasdedc
5         uiop
Name: A, dtype: object

In [65]:
listo=[]
for i in range(len(dfr)):
  listo.append(dfr.iloc[i,0])

print(listo)

['qwerty', 'uiop', 'wasd', 'fghjklzxc', 'zxcasdedc', 'uiop']


In [66]:
dfr['B'].value_counts()

be    3
bo    1
b     1
bi    1
Name: B, dtype: int64

In [67]:
nlp=spacy.load('en_core_web_sm')
text = ("When Sebastian Thrun started working on self-driving cars at "
        "Google in 2007, few people outside of the company took him "
        "seriously. “I can tell you very senior CEOs of major American "
        "car companies would shake my hand and turn away because I wasn’t "
        "worth talking to,” said Thrun, in an interview with Recode earlier "
        "this week.")
print(type(text))


for ent in nlp(text).ents:
  print(ent.text,ent.label_)

<class 'str'>
Sebastian NORP
Google ORG
2007 DATE
American NORP
Recode ORG
earlier this week DATE


In [68]:
dfr['A'].apply(lambda x: x+': test')

0       qwerty: test
1         uiop: test
2         wasd: test
3    fghjklzxc: test
4    zxcasdedc: test
5         uiop: test
Name: A, dtype: object

In [69]:
dfr['A'].apply(lambda x: [len(x) for i in range(len(x))])

0             [6, 6, 6, 6, 6, 6]
1                   [4, 4, 4, 4]
2                   [4, 4, 4, 4]
3    [9, 9, 9, 9, 9, 9, 9, 9, 9]
4    [9, 9, 9, 9, 9, 9, 9, 9, 9]
5                   [4, 4, 4, 4]
Name: A, dtype: object

In [70]:
dfr['A'].apply(lambda x: ["e" if i%2==0 else "o"  for i in range(len(x))])

0             [e, o, e, o, e, o]
1                   [e, o, e, o]
2                   [e, o, e, o]
3    [e, o, e, o, e, o, e, o, e]
4    [e, o, e, o, e, o, e, o, e]
5                   [e, o, e, o]
Name: A, dtype: object

In [71]:
dfr['e/o 1']=dfr['A'].apply(lambda x: ["e" if i%2==0 else "o"  for i in range(len(x))])
dfr

,A,B,C,D,E,e/o 1
0,qwerty,be,"mumbai, india",d,e,"[e, o, e, o, e, o]"
1,uiop,be,canada,None,e,"[e, o, e, o]"
2,wasd,bi,dallas tx,d,e,"[e, o, e, o]"
3,fghjklzxc,b,european union,d,e,"[e, o, e, o, e, o, e, o, e]"
4,zxcasdedc,bo,"kolkata, india",de,ei,"[e, o, e, o, e, o, e, o, e]"
5,uiop,be,orissa,dio,eu,"[e, o, e, o]"


In [72]:
dfr['e/o 2']=dfr['A'].apply(lambda x: [(i,"e") if i%2==0 else (i,"o")  for i in range(len(x))])
dfr

,A,B,C,D,E,e/o 1,e/o 2
0,qwerty,be,"mumbai, india",d,e,"[e, o, e, o, e, o]","[(0, e), (1, o), (2, e), (3, o), (4, e), (5, o)]"
1,uiop,be,canada,None,e,"[e, o, e, o]","[(0, e), (1, o), (2, e), (3, o)]"
2,wasd,bi,dallas tx,d,e,"[e, o, e, o]","[(0, e), (1, o), (2, e), (3, o)]"
3,fghjklzxc,b,european union,d,e,"[e, o, e, o, e, o, e, o, e]","[(0, e), (1, o), (2, e), (3, o), (4, e), (5, o..."
4,zxcasdedc,bo,"kolkata, india",de,ei,"[e, o, e, o, e, o, e, o, e]","[(0, e), (1, o), (2, e), (3, o), (4, e), (5, o..."
5,uiop,be,orissa,dio,eu,"[e, o, e, o]","[(0, e), (1, o), (2, e), (3, o)]"


In [73]:
dfr['e/o 3']=dfr['A'].apply(lambda x: [(i,"e") if i%2==0 else ""  for i in range(len(x))])
dfr

,A,B,C,D,E,e/o 1,e/o 2,e/o 3
0,qwerty,be,"mumbai, india",d,e,"[e, o, e, o, e, o]","[(0, e), (1, o), (2, e), (3, o), (4, e), (5, o)]","[(0, e), , (2, e), , (4, e), ]"
1,uiop,be,canada,None,e,"[e, o, e, o]","[(0, e), (1, o), (2, e), (3, o)]","[(0, e), , (2, e), ]"
2,wasd,bi,dallas tx,d,e,"[e, o, e, o]","[(0, e), (1, o), (2, e), (3, o)]","[(0, e), , (2, e), ]"
3,fghjklzxc,b,european union,d,e,"[e, o, e, o, e, o, e, o, e]","[(0, e), (1, o), (2, e), (3, o), (4, e), (5, o...","[(0, e), , (2, e), , (4, e), , (6, e), , (8, e)]"
4,zxcasdedc,bo,"kolkata, india",de,ei,"[e, o, e, o, e, o, e, o, e]","[(0, e), (1, o), (2, e), (3, o), (4, e), (5, o...","[(0, e), , (2, e), , (4, e), , (6, e), , (8, e)]"
5,uiop,be,orissa,dio,eu,"[e, o, e, o]","[(0, e), (1, o), (2, e), (3, o)]","[(0, e), , (2, e), ]"


In [74]:
dfr.to_csv('df.csv')

!ls

covid_csv.csv  df.csv  drive  sample_data


In [75]:
# gmaps=googlemaps.Client(key=config.get('gmaps','gApiKey'))
# geocode_result=gmaps.geocode(df.iloc[2,2])

# print(geocode_result)
# print(geocode_result[0])
# print(geocode_result[0]['formatted_address'])
# print(geocode_result[0]['formatted_address'].split(',')[-1])
# print(geocode_result[0]['formatted_address'].split(',')[-1].strip())


In [76]:
trial="  asd  "
print(not trial.strip())
print(not( not trial.strip()))

trial="    "
print(not trial.strip())
print(not( not trial.strip()))

False
True
True
False


In [77]:
class StreamListner(tweepy.StreamListner);
def on_status(self,status):
  print(status)
def on_error(self,error,    ):
  print(error)
def on_data(self,data):


SyntaxError: ignored

In [ ]:
covid_confirmed_df=pd.read_csv("https://github.com/CSSEGISandData/COVID-19/blob/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv")